<a href="https://colab.research.google.com/github/jespimentel/acidentes_piracicaba/blob/master/acidentes_piracicaba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Georreferenciamento e análise do banco de sentenças do TJSP

## Acidentes de trânsito em Piracicaba

### José Eduardo de Souza Pimentel (Jan/2026)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
# Importações necessárias

import os
import json
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from openai import OpenAI

In [4]:
# Configurações

MODELO = "gpt-5-mini" #"gpt-5-nano"
PATH_SENTENCAS = "/content/drive/MyDrive/Promotoria/Acidentes de trânsito"
MAX_INTERACOES = 10
CSV_SENTENCAS = '/content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas.csv'
CSV_SENTENCAS_ANALISADAS = '/content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas_analisadas.csv'
CSV_SENTENCAS_COM_ENDERECOS = '/content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas_com_enderecos.csv'
CSV_SENTENCAS_GEORREFERENCIADAS = '/content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas_georreferenciadas.csv'

# Limites geográficos (cantos do retângulo)
PONTO_1 = (-22.59505, -47.87594)
PONTO_2 = (-22.86262, -47.41858)

In [5]:
# Parametrização da requisição ao TJSP

PARAMS_TJSP = params = {
    "conversationId": "",
    "dadosConsulta.pesquisaLivre": "",
    "tipoNumero": "UNIFICADO",
    "numeroDigitoAnoUnificado": "",
    "foroNumeroUnificado": "",
    "dadosConsulta.nuProcesso": "",
    "dadosConsulta.nuProcessoAntigo": "",
    "classeTreeSelection.values": "",
    "classeTreeSelection.text": "",
    "assuntoTreeSelection.values": "3632",
    "assuntoTreeSelection.text": "",
    "agenteSelectedEntitiesList": "",
    "contadoragente": "0",
    "contadorMaioragente": "0",
    "cdAgente": "",
    "nmAgente": "",
    "dadosConsulta.dtInicio": "",
    "dadosConsulta.dtFim": "",
    "varasTreeSelection.values": "451-10,451-8,451-7101,451-7084,451-7094,451-7095,451-9,451-5623",
    "varasTreeSelection.text": "8 Registros selecionados",
    "dadosConsulta.ordenacao": "DESC"
}

# Download das sentenças

In [6]:
# Cria uma sessão persistente
session = requests.Session()

# 1a requisição para obter os cookies
response = session.get("https://esaj.tjsp.jus.br/cjpg/pesquisar.do", params=PARAMS_TJSP)

if response.status_code == 200:
    for i in range(1, MAX_INTERACOES + 1):
        pagina = str(i)
        url_pagina = f"https://esaj.tjsp.jus.br/cjpg/trocarDePagina.do?pagina={pagina}&conversationId="

        # Usa a mesma sessão para fazer a requisição
        response = session.get(url_pagina)

        if response.status_code == 200:
            # Salva o conteúdo da página
            with open(f"{PATH_SENTENCAS}/pagina_{pagina}.html", "wb") as file:
                file.write(response.content)
            print(f"Página {pagina} salva com sucesso.")
        else:
            print(f"Erro ao acessar a página {pagina}: {response.status_code}")

        # Intervalo entre requisições
        time.sleep(1)
else:
    print("Erro ao acessar a página inicial.")

Página 1 salva com sucesso.
Página 2 salva com sucesso.
Página 3 salva com sucesso.
Página 4 salva com sucesso.
Página 5 salva com sucesso.
Página 6 salva com sucesso.
Página 7 salva com sucesso.
Página 8 salva com sucesso.
Página 9 salva com sucesso.
Página 10 salva com sucesso.


# Criação do dataframe

In [7]:
def extrair_informacoes_html(html_path):
    processos = []
    with open(html_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        textos = soup.find_all(class_="fundocinza1")
        for texto in textos:
            processo = {}
            # Número do Processo
            processo['numero_do_processo'] = texto.find('span', class_='fonteNegrito').text.strip()

            # Classe, Assunto, Magistrado, Comarca, Foro, Vara, Data de Disponibilização
            for row in texto.find_all('tr', class_='fonte'):
                if row.find('strong'):
                    key = row.find('strong').text.strip().replace(':', '').lower().replace(' ', '_')
                    value = row.find('strong').next_sibling.strip()
                    processo[key] = value

            # Sentença
            sentenca_divs = texto.find_all('div', align='justify')
            sentenca = ''
            for div in sentenca_divs:
                span = div.find('span')
                if span:
                    sentenca += span.text.strip() + '\n'
            processo['sentenca'] = sentenca.strip()

            processos.append(processo)

        return processos

In [8]:
data = []

for filename in os.listdir(PATH_SENTENCAS):
    if filename.endswith(".html"):
        filepath = os.path.join(PATH_SENTENCAS, filename)
        data += (extrair_informacoes_html(filepath))

# Cria o DataFrame
df = pd.DataFrame(data)
df.head(5)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca
0,1500571-62.2024.8.26.0451,Inquérito Policial,Crimes de Trânsito,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,05/11/2025,SENTENÇA\n\nProcesso nº:\t1500571-62.2024.8.26...
1,1502202-19.2023.8.26.0599,Ação Penal - Procedimento Sumário,Crimes de Trânsito,Flavia de Cassia Gonzales de Oliveira,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,04/11/2025,
2,1502522-67.2019.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,NaN,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,31/10/2025,
3,1501240-52.2023.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,31/10/2025,SENTENÇA\n\nProcesso Digital nº:\t1501240-52.2...
4,0013813-80.2015.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,NaN,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,30/10/2025,


In [9]:
# Cria o dataframe com sentenças não vazias e salva em CSV
df_1 = df[(df['sentenca']!='')]
df_1 = df_1.drop_duplicates(subset=['numero_do_processo'], keep='last') # Despreza os processos repetidos
df_1.reset_index(drop=True, inplace=True)
df_1.to_csv(CSV_SENTENCAS, index=False, encoding='utf-8-sig')  # utf-8-sig para compatibilidade com Excel
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS}")

Arquivo CSV salvo em: /content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas.csv


# Consulta à API OpenAI para análise das sentenças

In [10]:
# Análisar com a LLM

SYSTEM_MSG = (
    "Você é assessor jurídico especializado em direito penal. "
    "Responda exclusivamente em JSON válido, sem comentários, com as chaves: "
    "'data_fato', 'endereco_fato', 'conduta_culposa'."
)

USER_TEMPLATE = (
    "Analise o texto a seguir e devolva um objeto JSON contendo as chaves já indicadas."
    "Para a chave 'data_fato' responda apenas com a data do fato (ex.: 24/03/2025)"
    "ou escreva 'Não se aplica'."
    "Para a chave 'endereco_fato' responda com o endereço principal do fato "
    "(ex.: rua Alvorada, 123, bairro Jardim Brasília, Piracicaba/SP)."
    "ou escreva 'Não se aplica'."
    "Para a chave 'conduta_culposa', resuma em até 100 palavras o porquê de ter havido o acidente, com "
    "enfase na conduta culposa. "
    "ou escreva 'Não se aplica'."
    "RESTRIÇÕES: para todos os casos, não mencione nomes. Use as iniciais dos envolvidos, "
    "caso precise referenciá-los.\n\n"
    "TEXTO FORNECIDO PARA ANÁLISE:\n\n{texto_fornecido}"
)

client = OpenAI(api_key=OPENAI_API_KEY)

def analisar_sentenca(texto_fornecido: str) -> dict:
    resposta_api = None

    try:
        response = client.chat.completions.create(
            model=MODELO,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": SYSTEM_MSG},
                {"role": "user", "content": USER_TEMPLATE.format(texto_fornecido=texto_fornecido)}
            ]
        )

        if (
            response
            and response.choices
            and response.choices[0].message
            and response.choices[0].message.content
        ):
            resposta_api = response.choices[0].message.content.strip()
        else:
            resposta_api = "Erro: Resposta da API em formato inesperado."
            print(f"Resposta inesperada da API: {response}")

    except Exception as e:
        resposta_api = f"Erro ao processar com OpenAI: {e}"

    return resposta_api

In [11]:
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 1. Definimos a função de processamento individual
def processar_linha(row):
    n_processo = row['numero_do_processo']
    print(f"Analisando o processo {n_processo}...")

    # Chamada da sua função original da API
    resposta_texto = analisar_sentenca(row['sentenca'])

    try:
        analise = json.loads(resposta_texto)
    except Exception as e:
        print(f"Erro ao converter JSON no processo {n_processo}: {e}")
        print("Resposta bruta da API:", resposta_texto)
        analise = {
            "data_fato": "Não se aplica",
            "endereco_fato": "Não se aplica",
            "conduta_culposa": "Não se aplica",
        }

    return (n_processo, analise)

# 2. Execução Paralela
# max_workers define quantas requisições ocorrem simultaneamente
MAX_REQUISICOES_SIMULTANEAS = 10

with ThreadPoolExecutor(max_workers=MAX_REQUISICOES_SIMULTANEAS) as executor:
    # Mapeia a função para cada linha do DataFrame
    # list() força a execução de todos os processos
    lista_de_respostas = list(executor.map(processar_linha, [row for _, row in df_1.iterrows()]))

# Resultado: lista_de_respostas conterá as tuplas (n_processo, analise)

Analisando o processo 1502313-93.2022.8.26.0451...
Analisando o processo 0009883-88.2014.8.26.0451...
Analisando o processo 1502947-89.2022.8.26.0451...
Analisando o processo 1502796-60.2021.8.26.0451...
Analisando o processo 0014888-04.2008.8.26.0451...
Analisando o processo 1500571-62.2024.8.26.0451...
Analisando o processo 1501240-52.2023.8.26.0451...
Analisando o processo 0002547-33.2014.8.26.0451...
Analisando o processo 1544390-88.2020.8.26.0451...
Analisando o processo 1512031-80.2023.8.26.0451...
Analisando o processo 0033917-98.2012.8.26.0451...
Analisando o processo 1510468-51.2023.8.26.0451...
Analisando o processo 1502522-67.2019.8.26.0451...
Analisando o processo 1501608-32.2021.8.26.0451...
Analisando o processo 1502827-51.2019.8.26.0451...
Analisando o processo 1500311-26.2024.8.26.0599...
Analisando o processo 0002976-63.2015.8.26.0451...
Analisando o processo 0008586-12.2015.8.26.0451...
Analisando o processo 1515776-44.2018.8.26.0451...
Analisando o processo 1532574-1

In [12]:
# Cria um DataFrame a partir da lista de respostas
# A lista_de_respostas contém tuplas (numero_do_processo, dict_analise)
# Vamos expandir o dicionário em colunas do DataFrame

data_para_df = []
for numero_processo, analise_dict in lista_de_respostas:
  row_data = {'numero_do_processo': numero_processo}
  row_data.update(analise_dict) # Adiciona as chaves do dicionário como colunas
  data_para_df.append(row_data)

In [13]:
df_respostas = pd.DataFrame(data_para_df)
display(df_respostas.head(5))

df_respostas.to_csv(CSV_SENTENCAS_ANALISADAS, index=False, encoding='utf-8-sig')
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS_ANALISADAS}")

,numero_do_processo,data_fato,endereco_fato,conduta_culposa
0,1502313-93.2022.8.26.0451,13/10/2021,"Avenida Antônio Fazanaro, cruzamento com Rua E...",O condutor não observou o dever de cuidado ao ...
1,0009883-88.2014.8.26.0451,Não se aplica,Não se aplica,Não se aplica
2,1502947-89.2022.8.26.0451,Não se aplica,Não se aplica,Não se aplica
3,1502796-60.2021.8.26.0451,Não se aplica,Não se aplica,Não se aplica
4,0014888-04.2008.8.26.0451,Não se aplica,Não se aplica,Não se aplica


Arquivo CSV salvo em: /content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas_analisadas.csv


In [14]:
df_geral = pd.merge(df_1, df_respostas, on='numero_do_processo')
df_geral.head(5)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca,data_fato,endereco_fato,conduta_culposa
0,1502313-93.2022.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Rodrigo Pares Andreucci,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,22/10/2025,SENTENÇA\n\nProcesso Digital nº:\t1502313-93.2...,13/10/2021,"Avenida Antônio Fazanaro, cruzamento com Rua E...",O condutor não observou o dever de cuidado ao ...
1,0009883-88.2014.8.26.0451,Ação Penal - Procedimento Sumário,Crimes de Trânsito,Flavia de Cassia Gonzales de Oliveira,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,09/10/2025,SENTENÇA\n\nProcesso nº:\t0009883-88.2014.8.26...,Não se aplica,Não se aplica,Não se aplica
2,1502947-89.2022.8.26.0451,Ação Penal - Procedimento Sumário,Crimes de Trânsito,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,06/10/2025,SENTENÇA\n\nProcesso nº:\t1502947-89.2022.8.26...,Não se aplica,Não se aplica,Não se aplica
3,1502796-60.2021.8.26.0451,Ação Penal - Procedimento Sumário,Crimes de Trânsito,GABRIELA DE ALMEIDA VERGUEIRO,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,21/11/2025,SENTENÇA\n\nProcesso nº:\t1502796-60.2021.8.26...,Não se aplica,Não se aplica,Não se aplica
4,0014888-04.2008.8.26.0451,Ação Penal - Procedimento Sumário,Crimes de Trânsito,Rodrigo Pares Andreucci,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,19/11/2025,SENTENÇA\n\nProcesso nº:\t0014888-04.2008.8.26...,Não se aplica,Não se aplica,Não se aplica


In [15]:
df_geral_com_endereco = df_geral[df_geral['endereco_fato'] != 'Não se aplica']
print(len(df_geral_com_endereco))

df_geral_com_endereco.to_csv(CSV_SENTENCAS_COM_ENDERECOS, index=False, encoding='utf-8-sig')
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS_COM_ENDERECOS}")

115
Arquivo CSV salvo em: /content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas_com_enderecos.csv


# Georreferenciamento do dataframe

In [16]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=8cdfa64ffcf407c33222923a4f9bbc302f7290c57d7814f921dd34fa38257cc5
  Stored in directory: /root/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


In [17]:
# Georreferenciar com o Google
import googlemaps
from google.colab import userdata

def buscar_coords(endereco):
    if not endereco:
        return None, None

    gmaps = googlemaps.Client(key=userdata.get('geo_google'))
    geocode_result = gmaps.geocode(endereco)

    if geocode_result:
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        return lat, lng
    else:
        print("Endereço não encontrado.")
        return None, None

In [18]:
df_geral_com_endereco[['latitude', 'longitude']] = df_geral_com_endereco['endereco_fato'].apply(
    lambda x: pd.Series(buscar_coords(x))
)

/tmp/ipython-input-419336096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral_com_endereco[['latitude', 'longitude']] = df_geral_com_endereco['endereco_fato'].apply(
/tmp/ipython-input-419336096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral_com_endereco[['latitude', 'longitude']] = df_geral_com_endereco['endereco_fato'].apply(


In [19]:
df_geral_com_endereco.head(5)

,numero_do_processo,classe,assunto,magistrado,comarca,foro,vara,data_de_disponibilização,sentenca,data_fato,endereco_fato,conduta_culposa,latitude,longitude
0,1502313-93.2022.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Rodrigo Pares Andreucci,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,22/10/2025,SENTENÇA\n\nProcesso Digital nº:\t1502313-93.2...,13/10/2021,"Avenida Antônio Fazanaro, cruzamento com Rua E...",O condutor não observou o dever de cuidado ao ...,-22.740622,-47.644175
6,1501240-52.2023.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,31/10/2025,SENTENÇA\n\nProcesso Digital nº:\t1501240-52.2...,09/12/2020,"Rodovia SP-304, Bairro Santa Terezinha, Piraci...",Não se aplica,-22.686229,-47.672379
11,1510468-51.2023.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Rodrigo Pares Andreucci,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,15/07/2025,SENTENÇA\n\nProcesso Digital nº:\t1510468-51.2...,21/10/2023,"Rua Santa Olímpia, estrada vicinal, bairro San...",O acusado (A.A.F.) conduzia veículo em velocid...,-22.614901,-47.744669
12,1502522-67.2019.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Gisela Ruffo,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,10/07/2025,SENTENÇA\n\nProcesso Digital nº:\t1502522-67.2...,23/12/2018,"Rodovia SP-147, Km 153, bairro Castelinho, Pir...","Colisão frontal em curva acentuada, em trecho ...",-22.713587,-47.653236
22,1509412-80.2023.8.26.0451,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Ana Claudia Madeira de Oliveira,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,29/04/2025,SENTENÇA\n\nProcesso Digital nº:\t1509412-80.2...,22/06/2023,"Rua Clara Nunes, 448, bairro Glebas Aliança, P...",O acidente decorreu da imprudência e negligênc...,-22.757028,-47.609179


In [20]:
# Salvar o DataFrame georreferenciado em um novo arquivo CSV
df_geral_com_endereco.to_csv(CSV_SENTENCAS_GEORREFERENCIADAS, index=False, encoding='utf-8-sig')
print(f"DataFrame de respostas salvo em: {CSV_SENTENCAS_GEORREFERENCIADAS}")

DataFrame de respostas salvo em: /content/drive/MyDrive/Promotoria/Acidentes de trânsito/sentencas_georreferenciadas.csv


In [21]:
import folium
from folium.plugins import MarkerCluster

# 2. Criar o mapa base (centralizado em uma coordenada média ou inicial)
mapa = folium.Map(location=[-22.7801, -47.66], zoom_start=10)

# 3. Criar o Cluster
marker_cluster = MarkerCluster().add_to(mapa)

# 4. Adicionar os marcadores ao cluster
for idx, row in df_geral_com_endereco.iterrows():
    # Só adiciona se houver coordenadas válidas
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        pop_up_text = f"<b>Processo:</b> {row['numero_do_processo']}<br><b>Conduta:</b> {row['conduta_culposa']}"

        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=folium.Popup(pop_up_text, max_width=300),
            tooltip=f"Processo {row['numero_do_processo']}"
        ).add_to(marker_cluster)

# Salvar ou exibir
mapa.save("mapa_processos_cluster.html")
mapa